# Video Processing for BT Lab
This notebook extracts metadata from video files 

<br>

![UofC logo](./pictures/uofc_logo-black.jpg)

In [ ]:
#import libraries
import os
import cv2
import json
import re
import json
import datetime

In [ ]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/record/JSON"

In [ ]:
def get_video_properties(mp4_file: str) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        old_fps = video.get(cv2.CAP_PROP_FPS)
        number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
        length = number_of_frames / old_fps
        video_time = datetime.timedelta(seconds=length)
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video_properties.append(video_time)
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    
    return video_properties

In [ ]:
def add_video_properties(metadata:dict) -> dict:
    new_patient_info = {}

    for patient_id, patient_info in metadata.items():
        try:
            video_path = patient_info["local path"]
            frames, fps, length_in_seconds, length_in_datetime = get_video_properties(video_path)
            patient_info["frames"] = frames
            patient_info["old fps"] = fps
            patient_info["length"] = length_in_seconds
            patient_info["length (datetime)"] = length_in_datetime
            new_patient_info[patient_id] = patient_info
        except Exception as e:
            print(f"{type(e)} - {patient_info['ID']}")

    return new_patient_info 

In [ ]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [ ]:
"""load JSON file """

metadata_rgb = load_json(json_dir, "rgb.json")
metadata_thermal = load_json(json_dir, "thermal.json")

In [ ]:
complete_metadata_rgb = add_video_properties(metadata_rgb)
complete_metadata_rgb